# Part IX — Deployment and Production Operations  
## 36. Production Readiness Checklist (The “Can We Deploy This?” Chapter)

This chapter is a professional, step-by-step checklist that turns a working Django
project into a production-ready service. It focuses on the things real teams check
before they deploy:

- security settings and secrets
- environment separation (dev/staging/prod)
- database readiness and migration strategy
- static and media strategy
- background jobs and realtime dependencies
- logging/health/readiness endpoints
- error handling and monitoring hooks
- operational documentation (runbooks)

By the end, you will have a **repeatable release checklist** you can apply to any
Django project.

---

## 36.0 Learning Outcomes

By the end of this chapter you should be able to:

1. Produce a production-ready settings configuration (no DEBUG, proper hosts,
   secure cookies, HTTPS).
2. Verify and document your environment variables and secrets.
3. Confirm database setup, backup plan, and migration run strategy.
4. Confirm static/media deployment plan (collectstatic, object storage decisions).
5. Confirm background workers (Celery) and realtime services (Redis) are ready.
6. Confirm observability:
   - logs
   - request IDs
   - health and readiness endpoints
   - error tracking plan
7. Confirm security posture:
   - CSRF, CORS, allowed hosts
   - headers, uploads, permissions
   - dependency updates
8. Create a “release runbook” and rollback plan.

---

# 36.1 Environment Separation (Dev / Staging / Production)

### 36.1.1 Industry standard expectation
- **Dev**: convenience, debug tools, local services
- **Staging**: production-like, used for final verification and migration tests
- **Production**: strict security, monitored, backed up

### 36.1.2 Settings modules (recommended)
You should have:

```text
config/settings/
  base.py
  dev.py
  prod.py
```

And set:

- dev: `DJANGO_SETTINGS_MODULE=config.settings.dev`
- prod: `DJANGO_SETTINGS_MODULE=config.settings.prod`

**Checklist**
- [ ] Settings are not “one file with a bunch of if DEBUG”.
- [ ] Secrets are loaded only from env vars.
- [ ] No dev-only packages are required in prod.

---

# 36.2 Secrets and Configuration (Must Be Explicit)

### 36.2.1 Required environment variables (minimum)
Create a table (put in README or docs) like:

- `DJANGO_SECRET_KEY`
- `DJANGO_DEBUG`
- `DJANGO_ALLOWED_HOSTS` (comma-separated)
- `DJANGO_CSRF_TRUSTED_ORIGINS` (comma-separated)
- `DATABASE_URL` or `POSTGRES_*` variables
- `SITE_URL`, `SITE_NAME`
- `CELERY_BROKER_URL`
- `CELERY_RESULT_BACKEND` (if used)
- `REDIS_URL` (if used for channels/cache)
- Email provider vars (SMTP host/user/pass) if used
- Webhook secrets (e.g., `WEBHOOK_PROVIDER_X_SECRET`)

**Checklist**
- [ ] No secrets committed to git.
- [ ] `.env` files are ignored.
- [ ] Secrets are rotated and can be replaced without code changes.

---

# 36.3 Debug and Error Pages

### 36.3.1 DEBUG must be False in production
**Checklist**
- [ ] `DEBUG=False` in prod settings.
- [ ] `ALLOWED_HOSTS` is correct.
- [ ] Custom error pages exist (404/500) and don’t leak stack traces.
- [ ] `ADMINS` and error email handling (optional) is configured if desired.

---

# 36.4 Allowed Hosts, CSRF Trusted Origins, CORS (if API used cross-origin)

### Checklist
- [ ] `ALLOWED_HOSTS` contains only real domains (no `*`).
- [ ] `CSRF_TRUSTED_ORIGINS` includes your HTTPS origins if behind proxy/CDN.
- [ ] If using cross-origin frontend:
  - [ ] `CORS_ALLOWED_ORIGINS` is explicit
  - [ ] credentials behavior is deliberate
  - [ ] `CORS_ALLOW_ALL_ORIGINS=False`

---

# 36.5 HTTPS and Secure Cookies

### Checklist (typical)
- [ ] `SECURE_SSL_REDIRECT=True` (if terminating TLS properly)
- [ ] `SECURE_PROXY_SSL_HEADER` configured if behind proxy
- [ ] `SESSION_COOKIE_SECURE=True`
- [ ] `CSRF_COOKIE_SECURE=True`
- [ ] `SESSION_COOKIE_HTTPONLY=True`
- [ ] SameSite configured:
  - [ ] `Lax` for same-site apps
  - [ ] `None` only if cross-origin cookie auth is required (and with Secure)

- [ ] HSTS configured carefully:
  - [ ] start small
  - [ ] increase after verification
  - [ ] include_subdomains/preload only when you’re sure

---

# 36.6 Static Files (collectstatic) — Production Strategy

### Checklist
- [ ] `STATIC_URL` set
- [ ] `STATIC_ROOT` set (production)
- [ ] `collectstatic` runs during build/deploy
- [ ] static files served by CDN/Nginx/platform static service (not Django)
- [ ] admin static files load correctly

**Verification**
- Open `/admin/` in production and confirm CSS loads.
- Confirm your main site CSS loads.

---

# 36.7 Media / Uploads — Persistence and Privacy

### Checklist
- [ ] `MEDIA_ROOT`/`MEDIA_URL` set for dev; production uses persistent storage
- [ ] public vs private media decision documented
- [ ] upload validation exists (size/type limits)
- [ ] media is backed up or stored in object storage
- [ ] local disk uploads won’t vanish on deploy (if using ephemeral hosts)

If you built exports:
- [ ] export files are stored in persistent storage
- [ ] download endpoints enforce authorization
- [ ] expired exports cleanup strategy exists

---

# 36.8 Database Readiness and Migration Plan

### Checklist
- [ ] Database is PostgreSQL in production (typical)
- [ ] DB credentials from env vars
- [ ] migrations plan documented:
  - [ ] when to run migrations (before/after deploy)
  - [ ] how to handle long migrations (expand/contract)
- [ ] backup strategy exists:
  - [ ] daily backups
  - [ ] retention
  - [ ] restore drill plan

**Release step checklist**
- [ ] run migrations
- [ ] verify app boot
- [ ] verify `/readyz` returns 200
- [ ] verify core pages and API endpoints

---

# 36.9 Background Tasks (Celery) and Redis Dependencies

If you use Celery:
- [ ] broker reachable from web and worker
- [ ] workers running with correct settings module and env vars
- [ ] task retries configured
- [ ] idempotency implemented for tasks that can duplicate (emails, webhooks)
- [ ] Celery beat running if you use periodic tasks
- [ ] worker monitoring/alerting planned

If you use Channels (WebSockets):
- [ ] ASGI server used in production (Uvicorn/Daphne)
- [ ] Redis channel layer configured
- [ ] Origin validation configured
- [ ] scale plan exists (multiple instances sharing Redis)

---

# 36.10 Observability and Monitoring

### Checklist
- [ ] request IDs enabled
- [ ] access logs emitted (method/path/status/duration/user_id)
- [ ] logs don’t contain secrets or auth headers
- [ ] error tracking configured (or plan exists)
- [ ] health endpoints:
  - [ ] `/healthz` (liveness)
  - [ ] `/readyz` (readiness with DB/cache check)
- [ ] alerting plan:
  - [ ] 5xx spike
  - [ ] latency spike
  - [ ] worker failure spike
  - [ ] DB down

---

# 36.11 Security Review Checklist (Pre-Launch)

### Checklist
- [ ] dependencies updated (Django security releases applied)
- [ ] `DEBUG=False`
- [ ] admin access restricted (staff only; MFA/SSO if possible)
- [ ] CSRF enforced
- [ ] CORS configured safely (if applicable)
- [ ] file uploads validated
- [ ] object-level permissions enforced and tested (tasks/org scoping)
- [ ] API rate limiting/throttling for abuse-sensitive endpoints
- [ ] security headers set (nosniff, frame options, etc.)
- [ ] secrets stored securely and rotated plan exists

---

# 36.12 Performance Readiness Checklist

### Checklist
- [ ] all list views paginated
- [ ] no N+1 in hot views (verified with debug toolbar + tests)
- [ ] indexes exist for common filters/orderings (verified with EXPLAIN in staging)
- [ ] caching strategy documented (what is cached, TTL, invalidation)
- [ ] export jobs are backgrounded (no long requests)

---

# 36.13 Deployment Documentation (Runbooks)

Every production app should have at least these docs:

### 36.13.1 Setup Runbook
- how to configure env vars
- how to start web
- how to start worker/beat
- how to run migrations
- how to seed initial admin user

### 36.13.2 Rollback Runbook
- how to revert code deployment
- what to do if migrations were applied
- how to disable feature flags (if used)
- how to restore DB backup (worst case)

### 36.13.3 Incident Response Quick Guide
- check `/readyz`
- check logs for request_id
- check worker health
- check DB health
- check Redis health
- temporarily enable maintenance mode (if available)
- escalate

---

# 36.14 Production Readiness “Gate” Script (Optional but Very Useful)

Create a simple management command that checks:
- required env vars exist
- migrations are applied
- DB connectivity works
- cache/redis connectivity works (if configured)

Example skeleton: `core/management/commands/healthcheck.py`:

```python
from django.core.management.base import BaseCommand
from django.db import connection


class Command(BaseCommand):
    help = "Run basic production health checks."

    def handle(self, *args, **options):
        with connection.cursor() as cursor:
            cursor.execute("SELECT 1")
            cursor.fetchone()
        self.stdout.write(self.style.SUCCESS("DB OK"))
```

You can run this in CI and during deploy.

---

# 36.15 Hands‑On Lab: Write Your Production Readiness Document

Create `docs/production-readiness.md` and fill out:

1. **Environment variables** (required/optional, examples)
2. **Deployment steps**:
   - build
   - migrate
   - collectstatic
   - start web
   - start worker/beat
3. **Health checks** and expected responses
4. **Rollback plan**
5. **Known risks**:
   - long migrations
   - external API dependencies
   - webhook retries
6. **Monitoring**:
   - what metrics/logs to watch
   - alerts

This document is what real teams use before “go live.”

---

## 36.16 Exercises (Do These Before Proceeding)

1. Ensure your project can start with `DEBUG=False` locally:
   - set env vars for SECRET_KEY and ALLOWED_HOSTS
   - run server and test `/healthz` and `/readyz`
2. Run `collectstatic` and confirm CSS loads from `STATIC_ROOT` when served by a simple static server (or just verify files exist).
3. Create a “deploy checklist” for your project and store it in `docs/`.
4. Add a CI check command to README:
   - `makemigrations --check --dry-run`
   - `ruff`, `black --check`, `pytest`, `migrate`

---

## 36.17 Chapter Summary

Production readiness is a checklist discipline:
- secure configuration
- correct environment separation
- DB + migrations plan
- static/media strategy
- background workers + redis dependencies
- observability
- runbooks

If you can produce these documents and checks, you can operate Django
professionally, not just “build features.”

---

Next chapter: **37. Deployment Options (Concepts + Practical Paths)**  
We’ll choose a deployment path and implement it end-to-end:
- VPS (Nginx + Gunicorn/Uvicorn)
- PaaS (Render/Heroku-like)
- Docker container deployment
including environment variables, static/media, database, workers, and rollouts.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../8. Frontend_integration/35. django_plus_frontend_frameworks.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='37. deployment_options.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
